# RAG 

#### Libraries

In [133]:
# processing:
import os
import re
import pandas as pd
from dotenv import load_dotenv

# neo4j:
from langchain_community.graphs import Neo4jGraph
from langchain_community.vectorstores import Neo4jVector

# llm:
from langchain.chat_models.gigachat import GigaChat
from langchain_community.embeddings import GigaChatEmbeddings
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# retrieval:
from langchain.chains import RetrievalQAWithSourcesChain
from langchain.memory import ConversationBufferMemory

#### .env

In [134]:
load_dotenv()

True

In [135]:
# neo4j:
NEO4J_URI = os.getenv('NEO4J_URI')
NEO4J_USERNAME = os.getenv('NEO4J_USERNAME')
NEO4J_PASSWORD = os.getenv('NEO4J_PASSWORD')
NEO4J_DATABASE= os.getenv('NEO4J_DATABASE')

# llm:
LLM_SCOPE = os.getenv('SCOPE')
LLM_AUTH = os.getenv('AUTH_DATA')

# telegram:
BOT_TOKEN = os.getenv('BOT_TOKEN')

#### Reference

In [136]:
reference = pd.read_excel('../docs/reference.xlsx')

## Setting Up LLM | GigaChat


[GigaChat](https://developers.sber.ru/docs/ru/gigachat/overview)

In [137]:
llm = GigaChat(credentials = LLM_AUTH, 
               temperature = 0.3,
               n = 1, 
               model = "GigaChat-Plus", # 32k context window
               repetition_penalty = 1.0,
               verify_ssl_certs = False)

In [138]:
parser = StrOutputParser()

In [139]:
embeddings = GigaChatEmbeddings(credentials = LLM_AUTH, verify_ssl_certs = False)

## Prompt Engineering

#### Prompt Template

In [140]:
template = """
            Задача: Анализировать заданный запрос и предоставлять детализированные ответы, опираясь на документы, правила и требования.

            Инструкции:

            1. При ответе на вопросы, особенно те, которые связаны с правилами или нормативными документами, активно ссылайтесь на номера пунктов, статей и разделов документов.
            2. Если требуемая информация отсутствует или вы не можете обнаружить необходимые пункты в документах, используйте фразу "Информация не найдена".
            3. Ваши ответы должны быть максимально точными и содержать не только ссылки на документы, но и объяснения их применения к данному запросу.

            Вопрос: {question}
            Контекст: {context}
            
            Ответ: Предоставьте ваш ответ, опираясь на указанные выше указания.
            """

prompt = ChatPromptTemplate.from_template(template)

#### Step Back Prompting

In [141]:
def step_back_prompt(model, parser, text):
    
    chain = model | parser

    template = f"""
                Задача: Анализировать представленные данные и вывести из них высокоуровневые концепции и основные принципы.

                Инструкции:
                Прежде чем приступить к абстракции, важно:
                1. Определить и выделить ключевые детали и специфику представленного материала.
                2. Анализировать связи между деталями для понимания более глубоких закономерностей и взаимосвязей.
                3. Структурировать полученные данные, выделяя общие элементы и паттерны.
                4. Формулировать высокоуровневые концепции и принципы, опираясь на проанализированные данные.

                Текст: \"{text}\"

                Проанализируйте и сформулируйте общие концепции и основные принципы на основе представленных деталей в виде развернутых вопросов.
                """

    model_response = chain.invoke(template)
    
    return model_response

#### Question Extraction

In [142]:
def extract_question(model, parser, text):
    
    chain = model | parser

    template = f"""
                            Мне нужно твоё содействие в анализе следующего делового письма.
                            Извлеки из него всю важную информацию для системы RAG. 
                            Нужны вопросы, ключевые параметры, и основные темы обсуждения. Вот текст письма:

                            Текст: \"{text}\"

                            Разбери письмо на следующие компоненты:

                            1. Все поставленные вопросы.
                            2. Перечень ключевых параметров, упомянутых в тексте.
                            3. Основные темы, которые обсуждаются.

                            Используй структурированный подход, чтобы я мог легко использовать эти данные для запросов в системе RAG.
                            Без дополнительных комментариев.
                            """

    model_response = chain.invoke(template)
    
    return model_response

## Setting Up Database | Neo4j

[Neo4j](https://workspace-preview.neo4j.io)

In [143]:
neo4j_vector = Neo4jVector.from_existing_index(
    embeddings,
    url = NEO4J_URI,
    username = NEO4J_USERNAME,
    password = NEO4J_PASSWORD,
    index_name = "vector"
    # search_type = 'hybrid'
)

## LLM Retrival Chain

[GigaChain](https://github.com/ai-forever/gigachain)

In [144]:
chain = RetrievalQAWithSourcesChain.from_chain_type(llm,
                                                    chain_type = "stuff", # "stuff", "map_rerank", "refine"
                                                    retriever = neo4j_vector.as_retriever(search_kwargs={"k": 5}),
                                                    return_source_documents = False,
                                                    reduce_k_below_max_tokens=False,
                                                    max_tokens_limit=32000,
                                                    chain_type_kwargs = {
                                                                        "verbose": False,
                                                                        "prompt": prompt, # step_back_prompt
                                                                        "document_variable_name": "context",
                                                                        "memory": ConversationBufferMemory(
                                                                            memory_key='history',
                                                                            input_key='question'),
                                                                        }
                                                    )

## Test

##### Reference Letters + Answers

In [145]:
letter = 5 # from the reference.xlsx num + 2

inquiry = str(reference['letter'][letter].replace('\n', ' '))
refer = str(reference['answer '][letter]).replace('\n', ' ')

abstraction = step_back_prompt(llm, parser, inquiry)
question = extract_question(llm, parser, inquiry)

##### Question Chain

In [146]:
print(f'Inquiry: {inquiry}\n')
print(f'Reference: {refer}\n')

print(f'Question: {question}\n')

chain.invoke(
            {"question": question},
            return_only_outputs = True
            )["answer"]


Inquiry: Добрый день! При определении НМЦ закупки услуг по поставке и монтажу здания (не капитальное строительство) какой метод определения НМЦ должен быть? 

Reference: В пункте 4.9.1. Положения о закупках предусмотрено, что "Проектно-сметный метод применяется для определения начальной (максимальной) цены договора (предмета закупки) на строительство, реконструкцию и капитальный ремонт объектов, а также может быть применен при определении начальной (максимальной) цены договора (предмета закупки) на текущий ремонт зданий, сооружений, строений, помещений".

Question: 1. Вопросы:
- Какой метод определения НМЦ должен быть при определении НМЦ закупки услуг по поставке и монтажу здания (не капитальное строительство)?

2. Ключевые параметры:
- Метод определения НМЦ

3. Основные темы обсуждения:
- Определение НМЦ закупки услуг по поставке и монтажу здания (не капитальное строительство)
- Выбор метода определения НМЦ



'Для определения начальной (максимальной) цены закупки услуг по поставке и монтажу здания (не капитальное строительство) необходимо использовать метод сопоставимых рыночных цен (анализ рынка). Этот метод является приоритетным и используется в большинстве случаев. Формула для определения НМЦ методом сопоставимых рыночных цен выглядит следующим образом: НМЦ=𝑣𝑛*𝑖=1𝑛∑Ц𝑖, где: НМЦ - начальная (максимальная) цена договора (предмета закупки), определяемая методом сопоставимых рыночных цен (анализ рынка);𝑣 - количество (объем) закупаемого товара (работы, услуги); 𝑛 - количество значений, используемых в расчете;𝑖 - номер источника ценовой информации; Ц𝑖 - цена единицы товара (работы, услуги), представленная в источнике с номером , скорректированная с учетом коэффициентов𝑖 (индексов), применяемых для пересчета цен товаров (работ, услуг) с учетом различий в характеристиках товаров, коммерческих и (или) финансовых условий поставок товаров (выполнения работ, оказания услуг), определяемых в соответс

In [147]:
docs_with_score_question = neo4j_vector.similarity_search_with_score(question, k = 5)

for doc, score in docs_with_score_question:
    print("-" * 100)
    print("Score: ", score)
    print(doc.page_content)
    print("-" * 100)

----------------------------------------------------------------------------------------------------
Score:  0.9446226954460144
4.5.13. Начальная (максимальная) цена договора (предмета закупки) методом  сопоставимых рыночных цен (анализ рынка) определяется по формуле: НМЦ=𝑣𝑛*𝑖=1𝑛∑Ц𝑖 где: НМЦ - начальная (максимальная) цена договора (предмета закупки), определяемая методом сопоставимых рыночных цен (анализ рынка);𝑣 - количество (объем) закупаемого товара (работы, услуги); 𝑛 - количество значений, используемых в расчете;𝑖 - номер источника ценовой информации; Ц𝑖 - цена единицы товара (работы, услуги), представленная в источнике с номером , скорректированная с учетом коэффициентов𝑖 (индексов), применяемых для пересчета цен товаров (работ, услуг) с учетом различий в характеристиках товаров, коммерческих и (или) финансовых условий поставок товаров (выполнения работ, оказания услуг), определяемых в соответствии с пунктом 4.5.10.
---------------------------------------------------------------

In [148]:
docs_with_score_question

[(Document(page_content='4.5.13. Начальная (максимальная) цена договора (предмета закупки) методом  сопоставимых рыночных цен (анализ рынка) определяется по формуле: НМЦ=𝑣𝑛*𝑖=1𝑛∑Ц𝑖 где: НМЦ - начальная (максимальная) цена договора (предмета закупки), определяемая методом сопоставимых рыночных цен (анализ рынка);𝑣 - количество (объем) закупаемого товара (работы, услуги); 𝑛 - количество значений, используемых в расчете;𝑖 - номер источника ценовой информации; Ц𝑖 - цена единицы товара (работы, услуги), представленная в источнике с номером , скорректированная с учетом коэффициентов𝑖 (индексов), применяемых для пересчета цен товаров (работ, услуг) с учетом различий в характеристиках товаров, коммерческих и (или) финансовых условий поставок товаров (выполнения работ, оказания услуг), определяемых в соответствии с пунктом 4.5.10.', metadata={'source': 'ПОЛОЖЕНИЕ/4. ОПРЕДЕЛЕНИЕ НАЧАЛЬНОЙ (МАКСИМАЛЬНОЙ) ЦЕНЫ  ДОГОВОРА (ПРЕДМЕТА ЗАКУПКИ)', 'bullet': '4.5.13'}),
  0.9446226954460144),
 (Document(p

##### Step Back Chain

In [149]:
print(f'Inquiry: {inquiry}\n')
print(f'Reference: {refer}\n')

print(f'Abstraction: {abstraction}\n')

chain.invoke(
            {"question": abstraction},
            return_only_outputs = True,
            )

Inquiry: Добрый день! При определении НМЦ закупки услуг по поставке и монтажу здания (не капитальное строительство) какой метод определения НМЦ должен быть? 

Reference: В пункте 4.9.1. Положения о закупках предусмотрено, что "Проектно-сметный метод применяется для определения начальной (максимальной) цены договора (предмета закупки) на строительство, реконструкцию и капитальный ремонт объектов, а также может быть применен при определении начальной (максимальной) цены договора (предмета закупки) на текущий ремонт зданий, сооружений, строений, помещений".

Abstraction: На основе представленных данных можно сформулировать следующие общие концепции и основные принципы:

1. При определении НМЦ закупки услуг по поставке и монтажу здания (не капитальное строительство) необходимо использовать метод определения НМЦ, который соответствует требованиям законодательства о государственных закупках.

2. Для определения НМЦ следует учитывать специфику и особенности объекта закупки, такие как тип здан

{'answer': 'На основе представленных данных можно сформулировать следующие общие концепции и основные принципы определения НМЦ закупки услуг по поставке и монтажу здания (не капитальное строительство):\n\n1. При определении НМЦ закупки услуг по поставке и монтажу здания необходимо использовать метод определения НМЦ, который соответствует требованиям законодательства о государственных закупках.\n\n2. Для определения НМЦ следует учитывать специфику и особенности объекта закупки, такие как тип здания, его размеры, материалы, используемые при строительстве, и другие факторы, которые могут повлиять на стоимость работ.\n\n3. При выборе метода определения НМЦ необходимо учитывать конкурентоспособность рынка, наличие достаточного количества предложений от потенциальных поставщиков, а также возможность проведения переговоров с ними для достижения оптимальной цены.\n\n4. Важно обеспечить прозрачность и объективность процесса определения НМЦ, чтобы избежать коррупционных рисков и гарантировать ра

In [150]:
docs_with_score_abstraction = neo4j_vector.similarity_search_with_score(abstraction, k = 5)

for doc, score in docs_with_score_abstraction:
    print("-" * 100)
    print("Score: ", score)
    print(doc.page_content)
    print("-" * 100)

----------------------------------------------------------------------------------------------------
Score:  0.9525815844535828
к  Заказчика, минимизации налоговых рисков; отсутствия ограничения допуска к участию в закупке путем установления  неизмеряемых требований к участникам закупки; создания условий для конкуренции между хозяйствующими субъектами при  проведении закупок в целях получения лучших условий удовлетворения потребности по  параметрам цены, качества и сроков на соответствующем товарном рынке; обеспечения баланса издержек на организацию и проведение процедур  конкурентных закупок и ожидаемой экономической выгоды от их результатов.
----------------------------------------------------------------------------------------------------
----------------------------------------------------------------------------------------------------
Score:  0.9519273042678833
1.1.3. Настоящее Положение регулирует закупочную деятельность ПАО «Компания  1» и Обществ Компания 1, устанавливает еди

In [151]:
docs_with_score_abstraction

[(Document(page_content='к  Заказчика, минимизации налоговых рисков; отсутствия ограничения допуска к участию в закупке путем установления  неизмеряемых требований к участникам закупки; создания условий для конкуренции между хозяйствующими субъектами при  проведении закупок в целях получения лучших условий удовлетворения потребности по  параметрам цены, качества и сроков на соответствующем товарном рынке; обеспечения баланса издержек на организацию и проведение процедур  конкурентных закупок и ожидаемой экономической выгоды от их результатов.', metadata={'source': 'ПОЛОЖЕНИЕ/1. ОБЩИЕ ПОЛОЖЕНИЯ', 'bullet': '1.1.3'}),
  0.9525815844535828),
 (Document(page_content='1.1.3. Настоящее Положение регулирует закупочную деятельность ПАО «Компания  1» и Обществ Компания 1, устанавливает единые правила и порядок закупок товаров,  работ, услуг, в том числе в целях коммерческого использования (далее - закупки),  направленные на реализацию следующих принципов закупок: создание условий для своевремен

## Response 

##### Temporary solution:

In [152]:
context = [doc[0].page_content for doc in docs_with_score_abstraction + docs_with_score_question]

In [153]:
context

['к  Заказчика, минимизации налоговых рисков; отсутствия ограничения допуска к участию в закупке путем установления  неизмеряемых требований к участникам закупки; создания условий для конкуренции между хозяйствующими субъектами при  проведении закупок в целях получения лучших условий удовлетворения потребности по  параметрам цены, качества и сроков на соответствующем товарном рынке; обеспечения баланса издержек на организацию и проведение процедур  конкурентных закупок и ожидаемой экономической выгоды от их результатов.',
 '1.1.3. Настоящее Положение регулирует закупочную деятельность ПАО «Компания  1» и Обществ Компания 1, устанавливает единые правила и порядок закупок товаров,  работ, услуг, в том числе в целях коммерческого использования (далее - закупки),  направленные на реализацию следующих принципов закупок: создание условий для своевременного и полного обеспечения потребностей в  товарах, работах, услугах и выявления поставщика (подрядчика, исполнителя),  способного своевременн

In [154]:
template = f"""
            Задача: Анализировать заданный запрос и предоставлять детализированные ответы, опираясь на документы, правила и требования.

            Инструкции:

            1. При ответе на вопросы, особенно те, которые связаны с правилами или нормативными документами, активно ссылайтесь на номера пунктов, статей и разделов документов.
            2. Если требуемая информация отсутствует или вы не можете обнаружить необходимые пункты в документах, используйте фразу "Информация не найдена".
            3. Ваши ответы должны быть максимально точными и содержать не только ссылки на документы, но и объяснения их применения к данному запросу.

            Вопрос: {question}
            Контекст: {context}
            
            Ответ: Предоставьте ваш ответ, опираясь на указанные выше указания.
            """


In [155]:
chains = llm | parser
chains.invoke(template)

'Для определения начальной максимальной цены (НМЦ) закупки услуг по поставке и монтажу здания (не капитальное строительство) необходимо использовать метод сопоставимых рыночных цен (анализ рынка). Данный метод является приоритетным и используется в большинстве случаев. Для определения НМЦ следует использовать формулу, указанную в пункте 4.5.13: НМЦ=𝑣𝑛*𝑖=1𝑛∑Ц𝑖, где 𝑣 - количество (объем) закупаемого товара (работы, услуги); 𝑛 - количество значений, используемых в расчете; 𝑖 - номер источника ценовой информации; Ц𝑖 - цена единицы товара (работы, услуги), представленная в источнике с номером , скорректированная с учетом коэффициентов𝑖 (индексов), применяемых для пересчета цен товаров (работ, услуг) с учетом различий в характеристиках товаров, коммерческих и (или) финансовых условий поставок товаров (выполнения работ, оказания услуг), определяемых в соответствии с пунктом 4.5.10.'

## Telegram

In [156]:
import telebot
from time import sleep

In [157]:
user_conversations = {}
bot = telebot.TeleBot(BOT_TOKEN)

In [158]:
question = extract_question(llm, parser, inquiry)

chain = RetrievalQAWithSourcesChain.from_chain_type(llm,
                                                    chain_type = "stuff", # "stuff", "map_rerank", "refine"
                                                    retriever = neo4j_vector.as_retriever(search_kwargs={"k": 5}),
                                                    return_source_documents = False,
                                                    reduce_k_below_max_tokens=False,
                                                    max_tokens_limit=32000,
                                                    chain_type_kwargs = {
                                                                        "verbose": False,
                                                                        "prompt": prompt, # step_back_prompt
                                                                        "document_variable_name": "context",
                                                                        "memory": ConversationBufferMemory(
                                                                            memory_key='history',
                                                                            input_key='question'),
                                                                        }
                                                    )

@bot.message_handler(content_types=['audio', 'video', 'document', 'photo', 'sticker', 'voice', 'location', 'contact'])
def not_text(message):
    user_id = message.chat.id
    bot.send_message(user_id, 'Я работаю только с текстовыми сообщениями!')

@bot.message_handler(content_types=['text'])
def handle_text_message(message):
    user_id = message.chat.id
    question = extract_question(llm, parser, message.text)

    response = chain.invoke(
            {"question": question},
            return_only_outputs = True
            )["answer"]

    bot.send_message(user_id, response)
    sleep(2)

In [159]:
bot.polling(none_stop=True)

## Basic Queries | Neo4j

[Cypher](https://neo4j.com/docs/cypher-cheat-sheet/5/auradb-enterprise)

In [160]:
graph = Neo4jGraph(url = NEO4J_URI, username = NEO4J_USERNAME, password = NEO4J_PASSWORD, database = NEO4J_DATABASE)

In [161]:
print(graph.schema)

Node properties are the following:
Chunk {embedding: LIST, id: STRING, text: STRING, source: STRING, bullet: STRING}
Relationship properties are the following:

The relationships are the following:



In [162]:
cypher = """
  SHOW VECTOR INDEXES
  """
graph.query(cypher)

[{'id': 5,
  'name': 'vector',
  'state': 'ONLINE',
  'populationPercent': 100.0,
  'type': 'VECTOR',
  'entityType': 'NODE',
  'labelsOrTypes': ['Chunk'],
  'properties': ['embedding'],
  'indexProvider': 'vector-2.0',
  'owningConstraint': None,
  'lastRead': neo4j.time.DateTime(2024, 4, 16, 13, 21, 38, 725000000, tzinfo=<UTC>),
  'readCount': 43}]

In [163]:
cypher = """
  MATCH (n)
  RETURN count(n)
  """
graph.query(cypher)

[{'count(n)': 1030}]

In [170]:
cypher = """
    MATCH (n:Chunk {bullet: "1.3.1"})
    RETURN n.text AS text
    """
graph.query(cypher)

[{'text': '1.3.1. Планирование закупок ПАО «Компания 1» и Обществ Компания 1, в рамках  которого:'}]

In [165]:
highest_scored_content = str(neo4j_vector.similarity_search(inquiry, k=3))

match = re.search(r'\d+(\.\d+)+', highest_scored_content)

if match:
    highest_scored_document_number = match.group(0)
    print("Highest scored document number:", highest_scored_document_number)

else:
    print("No matching number found")

if match:
    cypher_query = f"""
                    MATCH (n:Chunk)
                    WHERE n.bullet STARTS WITH '{highest_scored_document_number}'
                    RETURN n.text
                    """

Highest scored document number: 4.5.13


In [166]:
graph.query(cypher_query)

[{'n.text': '4.5.13. Начальная (максимальная) цена договора (предмета закупки) методом  сопоставимых рыночных цен (анализ рынка) определяется по формуле: НМЦ=𝑣𝑛*𝑖=1𝑛∑Ц𝑖 где: НМЦ - начальная (максимальная) цена договора (предмета закупки), определяемая методом сопоставимых рыночных цен (анализ рынка);𝑣 - количество (объем) закупаемого товара (работы, услуги); 𝑛 - количество значений, используемых в расчете;𝑖 - номер источника ценовой информации; Ц𝑖 - цена единицы товара (работы, услуги), представленная в источнике с номером , скорректированная с учетом коэффициентов𝑖 (индексов), применяемых для пересчета цен товаров (работ, услуг) с учетом различий в характеристиках товаров, коммерческих и (или) финансовых условий поставок товаров (выполнения работ, оказания услуг), определяемых в соответствии с пунктом 4.5.10.'}]